In [61]:
#pip install catboost

In [63]:
#pip install optuna

In [2]:
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
import prince
import optuna
import tensorflow as tf

from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, GridSearchCV, RandomizedSearchCV, validation_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder, PolynomialFeatures, StandardScaler, SplineTransformer, KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, f1_score, roc_curve, auc, recall_score, precision_recall_curve, make_scorer
from sklearn.kernel_approximation import Nystroem
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.utils.class_weight import compute_class_weight

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.metrics import classification_report_imbalanced

from prince import FAMD

from scipy.stats import chi2_contingency

from yellowbrick.classifier import DiscriminationThreshold

from catboost import CatBoostClassifier,Pool, cv
from catboost.utils import eval_metric

from optuna.samplers import TPESampler

In [3]:
file = open('../../../conf/global_conf.yml', 'r') 
conf = yaml.safe_load(file)
df = pd.read_csv(conf['local_data_path'] + "data_cleaned_final_sans_dummies.csv",index_col=0,low_memory=False)

In [4]:
print(df.shape)

(447136, 41)


In [5]:
data = df.drop(['grav_rec', 'an', 'jour', 'date', 'grav', 'dep'], axis=1)

y = df.grav
numerical_features = ['age_usager', 'mois', 'heure', 'lat', 'long']
categorical_features = [col for col in data.columns if col not in numerical_features]
for col in categorical_features:
    data[col] = data[col].astype('int')
data[categorical_features] = data[categorical_features].astype('object')

#data.info()

In [6]:
data.head()

,mois,lum,agg,int,atm,col,lat,long,catr,circ,...,eq_ceinture,eq_casque,eq_siege,eq_gilet,eq_airbag,eq_gants,eq_indetermine,eq_autre,jour_chome,prox_pt_choc
0,11,2,0,0,0,2,48.896210,2.470120,1,0,...,1,0,0,0,0,0,0,0,1,0
1,11,2,0,0,0,2,48.896210,2.470120,1,0,...,1,0,0,0,0,0,0,0,1,0
2,11,2,0,0,0,2,48.896210,2.470120,1,0,...,1,0,0,0,0,0,0,0,1,1
3,11,2,0,0,0,6,48.930700,2.368800,1,0,...,1,0,0,0,0,0,0,0,1,1
4,11,0,0,0,0,4,48.935872,2.319174,1,0,...,1,0,0,0,0,0,0,0,1,1


In [7]:
data_train, data_test, y_train, y_test = train_test_split(data, y, test_size=0.25, random_state=42, stratify=y)

In [8]:
class TransfoHour(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transfo = X.copy()
        X_transfo[f'{self.column_name}_sin'] = np.sin(X_transfo[self.column_name]*(2.*np.pi/24))
        X_transfo[f'{self.column_name}_cos'] = np.cos(X_transfo[self.column_name]*(2.*np.pi/24))
        X_transfo = X_transfo.drop(self.column_name, axis=1)
        return X_transfo
    
    def get_feature_names_out(self):
        pass
    
class TransfoMonth(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transfo = X.copy()
        X_transfo[f'{self.column_name}_sin'] = np.sin((X_transfo[self.column_name]-1)*(2.*np.pi/12))
        X_transfo[f'{self.column_name}_cos'] = np.cos((X_transfo[self.column_name]-1)*(2.*np.pi/12))
        X_transfo = X_transfo.drop(self.column_name, axis=1)
        return X_transfo

    def get_feature_names_out(self):
        pass
    

**Catboost**

In [8]:
preprocessor = ColumnTransformer(transformers=[
    ("transfo_heure", TransfoHour('heure'), ['heure']),
    ("transfo_mois", TransfoMonth('mois'), ['mois']),
    ("transfo_lat_long", RobustScaler(), ['lat', 'long']),
    ('transfo_age_usager', StandardScaler(), ['age_usager']),
    #("one_hot_encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ['place_rec','catr', 'surf', 'infra', 'situ', 'motor', 'manv', 'obsm', 'catv', 'col', 'lum'])
    ],
    remainder="passthrough"
).set_output(transform="pandas")

In [9]:
data_process = preprocessor.fit_transform(data_train)
categorical_numbers = [int(i) for i in range(len(data_process.columns)) if 'remainder' in data_process.columns[i]]
print(categorical_numbers)

[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]


In [10]:
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))
class_weights

{1.0: 0.6057177536467477,
 2.0: 9.161621680690635,
 3.0: 1.5969447037086422,
 4.0: 0.6196864536443666}

In [11]:
cv_dataset = Pool(data = data_process,
                  label = y_train,
                  cat_features = categorical_numbers,
                  weight = y_train.map(class_weights))

In [12]:
params = {"iterations": 500,
          "depth": 2,
          "loss_function": "MultiClassOneVsAll",
          "verbose": False}

In [15]:
scores = cv(cv_dataset, params, fold_count=2, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/2]


KeyboardInterrupt: 

**GridsearchCV**

In [13]:
data_process_train = preprocessor.fit_transform(data_train)

In [14]:
data__train, data__val, y__train, y__val = train_test_split(data_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [15]:
data_process__train = preprocessor.fit_transform(data__train)
train_dataset = Pool(data = data_process__train,
                     label = y__train,
                     cat_features = categorical_numbers,
                     weight = y__train.map(class_weights))

data_process__val = preprocessor.fit_transform(data__val)
eval_dataset = Pool(data = data_process__val,
                   label = y__val,
                   cat_features = categorical_numbers,
                   weight = y__val.map(class_weights))

In [29]:
clf = CatBoostClassifier(iterations=250, random_seed = 42, depth=6)
f1 = make_scorer(f1_score, average='micro')
params = {'l2_leaf_reg' : [2, 10, 20, 30],
          'loss_function': ["MultiClass"]}

clf_grid = GridSearchCV(estimator=clf, param_grid=params, cv=2, scoring=f1, return_train_score=True,verbose=10)
clf_grid.fit(data_process__train, y__train, cat_features=categorical_numbers, eval_set=(data_process__val, y__val))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2; 1/4] START l2_leaf_reg=2, loss_function=MultiClass.....................
0:	learn: 1.3589901	test: 1.3590563	best: 1.3590563 (0)	total: 643ms	remaining: 2m 40s
1:	learn: 1.3343016	test: 1.3343883	best: 1.3343883 (1)	total: 1.29s	remaining: 2m 40s
2:	learn: 1.3101547	test: 1.3102353	best: 1.3102353 (2)	total: 1.92s	remaining: 2m 38s
3:	learn: 1.2882002	test: 1.2882499	best: 1.2882499 (3)	total: 2.47s	remaining: 2m 31s
4:	learn: 1.2674111	test: 1.2674516	best: 1.2674516 (4)	total: 2.98s	remaining: 2m 26s
5:	learn: 1.2483993	test: 1.2484235	best: 1.2484235 (5)	total: 3.46s	remaining: 2m 20s
6:	learn: 1.2302394	test: 1.2301674	best: 1.2301674 (6)	total: 4.06s	remaining: 2m 20s
7:	learn: 1.2122223	test: 1.2121075	best: 1.2121075 (7)	total: 4.55s	remaining: 2m 17s
8:	learn: 1.1952914	test: 1.1951753	best: 1.1951753 (8)	total: 5.03s	remaining: 2m 14s
9:	learn: 1.1794437	test: 1.1792872	best: 1.1792872 (9)	total: 5.55s	remainin

GridSearchCV(cv=2,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000022D21D9C6D0>,
             param_grid={'l2_leaf_reg': [2, 10, 20, 30],
                         'loss_function': ['MultiClass']},
             return_train_score=True,
             scoring=make_scorer(f1_score, average=micro), verbose=10)

In [30]:
best_param = clf_grid.best_params_
best_param

{'l2_leaf_reg': 2, 'loss_function': 'MultiClass'}

In [31]:
cv_results = pd.DataFrame(clf_grid.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l2_leaf_reg,param_loss_function,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,167.012506,19.848514,1.390172,0.071396,2,MultiClass,"{'l2_leaf_reg': 2, 'loss_function': 'MultiClass'}",0.649992,0.651765,0.650878,0.000887,1,0.654246,0.652687,0.653467,0.000779
1,187.899845,4.541918,1.563004,0.086070,10,MultiClass,"{'l2_leaf_reg': 10, 'loss_function': 'MultiCla...",0.649944,0.651598,0.650771,0.000827,2,0.653856,0.652051,0.652954,0.000903
2,184.402123,12.703572,1.528816,0.196439,20,MultiClass,"{'l2_leaf_reg': 20, 'loss_function': 'MultiCla...",0.649666,0.650628,0.650147,0.000481,3,0.653085,0.650922,0.652003,0.001081
3,180.020046,3.122639,1.332927,0.034866,30,MultiClass,"{'l2_leaf_reg': 30, 'loss_function': 'MultiCla...",0.648934,0.650190,0.649562,0.000628,4,0.652258,0.650389,0.651324,0.000934


**CatBoostClassifier - prédictions**


In [19]:
GPU_AVAILABLE = tf.config.list_physical_devices('GPU')
task_type = "GPU" if GPU_AVAILABLE else "CPU"
print(task_type)

CPU


In [20]:

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations= 392,
                           depth= 8,
                           learning_rate= 0.1636393093068123,
                           l2_leaf_reg= 8.382816537101032,
                           grow_policy = "SymmetricTree",
                           loss_function= "MultiClass",
                           task_type = task_type)
# Fit model with `use_best_model=True`
model.fit(train_dataset,
          use_best_model=True,
          eval_set=eval_dataset,
          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.2969188	test: 1.2984045	best: 1.2984045 (0)	total: 2.31s	remaining: 15m 5s
1:	learn: 1.2363987	test: 1.2391679	best: 1.2391679 (1)	total: 4.73s	remaining: 15m 22s
2:	learn: 1.1903559	test: 1.1942341	best: 1.1942341 (2)	total: 6.98s	remaining: 15m 4s
3:	learn: 1.1570133	test: 1.1618803	best: 1.1618803 (3)	total: 8.83s	remaining: 14m 16s
4:	learn: 1.1309434	test: 1.1364592	best: 1.1364592 (4)	total: 11s	remaining: 14m 11s
5:	learn: 1.1084397	test: 1.1146958	best: 1.1146958 (5)	total: 13s	remaining: 13m 59s
6:	learn: 1.0900169	test: 1.0969369	best: 1.0969369 (6)	total: 15s	remaining: 13m 46s
7:	learn: 1.0758025	test: 1.0836575	best: 1.0836575 (7)	total: 17.6s	remaining: 14m 4s
8:	learn: 1.0635065	test: 1.0719516	best: 1.0719516 (8)	total: 20.2s	remaining: 14m 21s
9:	learn: 1.0512556	test: 1.0600879	best: 1.0600879 (9)	total: 22.8s	remaining: 14m 31s
10:	learn: 1.0420850	test: 1.0523242	best: 1.0523242 (10)	total: 25.5s	remaining: 14m 43s
11:	learn: 1.0346332	test: 1.0454512	be

In [21]:
print(model.get_all_params())

{'nan_mode': 'Min', 'eval_metric': 'MultiClass', 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=3:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1', 'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'], 'iterations': 392, 'sampling_frequency': 'PerTree', 'fold_permutation_block': 0, 'leaf_estimation_method': 'Newton', 'random_score_type': 'NormalWithModelSizeDecrease', 'counter_calc_method': 'SkipTest', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'ctr_leaf_count_limit': 18446744073709551615, 'bayesian_matrix_reg': 0.10000000149011612, 'one_hot_max_size': 2, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 8.382816314697266, 'random_strength': 1, 'rsm': 1, 'boost_from_average': False, 'max_ctr_complexity': 4, 'model_size_reg': 0.5, 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Unif

In [22]:
data_process_test = preprocessor.fit_transform(data_test)

In [23]:
print(model.score(data_process_train, y_train))
print(model.score(data_process_test, y_test))

0.6311457811493595
0.6149896228440564


In [24]:
preds = model.predict(data_process_test)
y_pred = [int(val[0]) for val in preds]
print(y_pred)

[1, 3, 4, 4, 1, 4, 4, 4, 1, 1, 4, 1, 2, 4, 1, 3, 3, 1, 4, 4, 1, 1, 1, 4, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 4, 3, 1, 1, 3, 1, 1, 1, 4, 1, 4, 1, 1, 4, 2, 2, 3, 3, 3, 1, 2, 1, 4, 4, 3, 1, 4, 2, 1, 1, 4, 2, 2, 4, 2, 4, 3, 1, 3, 1, 1, 1, 3, 3, 3, 3, 4, 4, 4, 3, 4, 4, 4, 4, 3, 2, 4, 4, 4, 2, 1, 3, 2, 3, 1, 4, 4, 1, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 1, 3, 3, 4, 2, 4, 2, 1, 4, 4, 4, 3, 4, 1, 1, 1, 1, 4, 1, 1, 3, 3, 1, 4, 2, 1, 2, 3, 1, 1, 4, 3, 4, 3, 1, 1, 1, 4, 1, 3, 1, 4, 3, 1, 1, 3, 1, 4, 1, 3, 2, 4, 1, 2, 2, 3, 2, 4, 2, 4, 4, 1, 3, 4, 1, 1, 4, 4, 1, 1, 3, 3, 1, 4, 4, 3, 4, 1, 1, 4, 3, 1, 1, 4, 3, 2, 3, 3, 4, 1, 4, 4, 1, 4, 4, 2, 3, 4, 1, 1, 1, 3, 1, 2, 2, 2, 3, 1, 1, 1, 1, 1, 3, 2, 1, 4, 1, 1, 1, 1, 1, 2, 1, 2, 1, 4, 4, 4, 4, 4, 1, 4, 3, 4, 4, 1, 1, 4, 1, 4, 4, 1, 1, 1, 4, 4, 4, 1, 4, 1, 1, 4, 4, 4, 4, 1, 4, 1, 1, 3, 4, 1, 1, 1, 1, 4, 1, 1, 1, 4, 1, 1, 1, 3, 4, 4, 1, 4, 1, 4, 1, 1, 1, 4, 4, 4, 2, 4, 1, 4, 4, 4, 1, 1, 1, 1, 3, 1, 1, 1, 3, 3, 1, 1, 3, 3, 4, 1, 4, 1, 2, 2, 1, 4, 3, 4, 

In [28]:
cr = classification_report_imbalanced(y_test, y_pred)
print(cr)
pd.crosstab(y_test, y_pred)

                   pre       rec       spe        f1       geo       iba       sup

        1.0       0.75      0.79      0.81      0.76      0.80      0.64     46137
        2.0       0.16      0.57      0.91      0.25      0.72      0.50      3050
        3.0       0.41      0.49      0.87      0.44      0.65      0.41     17500
        4.0       0.71      0.49      0.86      0.58      0.65      0.41     45097

avg / total       0.66      0.61      0.84      0.63      0.71      0.51    111784



col_0,1,2,3,4
grav,,,,
1.0,36223,1581,2576,5757
2.0,163,1743,914,230
3.0,1448,4463,8518,3071
4.0,10738,3202,8895,22262


**Optuna**

In [18]:

def objective(trial):
    params = {
        'iterations' : trial.suggest_int('iterations', 250, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0, 10),
        'grow_policy' : trial.suggest_categorical('grow_policy', ["SymmetricTree", "Lossguide"]),
        'loss_function' :  trial.suggest_categorical('loss_function', ["MultiClass", "MultiClassOneVsAll"])
    }

    model = CatBoostClassifier(**params,  random_seed=42)
    model.fit(train_dataset, eval_set=eval_dataset)
    preds = model.predict(eval_dataset)
    y_pred = [int(val[0]) for val in preds]
    cat_f1 = f1_score(eval_dataset.get_label(), y_pred, average = 'micro')
    return cat_f1

sampler = TPESampler(seed=123)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=8, n_jobs=-1)

[I 2024-05-02 09:59:31,131] A new study created in memory with name: no-name-392122fa-a1e7-4580-b22e-edbe63cd3a02


0:	learn: 0.6107279	test: 0.6107279	best: 0.6107279 (0)	total: 1.29s	remaining: 5m 35s
0:	learn: 0.6608735	test: 0.6608735	best: 0.6608735 (0)	total: 1.63s	remaining: 6m 54s
0:	learn: 0.6687676	test: 0.6687676	best: 0.6687676 (0)	total: 1.73s	remaining: 7m 16s
1:	learn: 0.5806263	test: 0.5806263	best: 0.5806263 (1)	total: 2.38s	remaining: 5m 8s
1:	learn: 0.6367997	test: 0.6367997	best: 0.6367997 (1)	total: 2.72s	remaining: 5m 43s
1:	learn: 0.6490854	test: 0.6490854	best: 0.6490854 (1)	total: 2.85s	remaining: 5m 57s
2:	learn: 0.5691324	test: 0.5691324	best: 0.5691324 (2)	total: 3.36s	remaining: 4m 49s
2:	learn: 0.6186944	test: 0.6186944	best: 0.6186944 (2)	total: 4.03s	remaining: 5m 38s
2:	learn: 0.6331203	test: 0.6331203	best: 0.6331203 (2)	total: 4.03s	remaining: 5m 35s
3:	learn: 0.6050103	test: 0.6050103	best: 0.6050103 (3)	total: 5.3s	remaining: 5m 32s
0:	learn: 1.3192988	test: 1.3198093	best: 1.3198093 (0)	total: 5.31s	remaining: 24m 3s
3:	learn: 0.6201273	test: 0.6201273	best: 0.6

[I 2024-05-02 10:21:30,090] Trial 0 finished with value: 0.5996803358858751 and parameters: {'iterations': 261, 'learning_rate': 0.41536760999256284, 'depth': 8, 'l2_leaf_reg': 9.834450683887164, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClassOneVsAll'}. Best is trial 0 with value: 0.5996803358858751.
[I 2024-05-02 10:23:56,530] Trial 2 finished with value: 0.5932393425415683 and parameters: {'iterations': 253, 'learning_rate': 0.10280021433144924, 'depth': 5, 'l2_leaf_reg': 0.21659708184573856, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClassOneVsAll'}. Best is trial 0 with value: 0.5996803358858751.
[I 2024-05-02 10:24:24,346] Trial 1 finished with value: 0.5980700875497984 and parameters: {'iterations': 255, 'learning_rate': 0.1387096134769567, 'depth': 5, 'l2_leaf_reg': 1.2606496677919155, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClassOneVsAll'}. Best is trial 0 with value: 0.5996803358858751.
[I 2024-05-02 10:25:40,637] Trial 6 finished with value: 0.598

In [25]:
def objective(trial):
    params = {
        'iterations' : trial.suggest_int('iterations', 250, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 10),
        'grow_policy' : trial.suggest_categorical('grow_policy', ["SymmetricTree", "Lossguide"]),
        'loss_function' :  trial.suggest_categorical('loss_function', ["MultiClass", "MultiClassOneVsAll"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0)
    }

    model = CatBoostClassifier(**params,  random_seed=42)
    model.fit(train_dataset, eval_set=eval_dataset)
    preds = model.predict(eval_dataset)
    y_pred = [int(val[0]) for val in preds]
    cat_f1 = f1_score(eval_dataset.get_label(), y_pred, average = 'micro')
    return cat_f1

sampler = TPESampler(seed=123)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=8, n_jobs=-1)

[I 2024-05-02 10:54:37,490] A new study created in memory with name: no-name-2e92b724-fe48-4384-b6bb-73d780075800


0:	learn: 0.6573906	test: 0.6573906	best: 0.6573906 (0)	total: 803ms	remaining: 3m 30s
0:	learn: 0.6647320	test: 0.6647320	best: 0.6647320 (0)	total: 1.15s	remaining: 5m 38s
1:	learn: 0.6316822	test: 0.6316822	best: 0.6316822 (1)	total: 1.77s	remaining: 3m 50s
1:	learn: 0.6426854	test: 0.6426854	best: 0.6426854 (1)	total: 2.03s	remaining: 4m 58s
2:	learn: 0.6130118	test: 0.6130118	best: 0.6130118 (2)	total: 2.46s	remaining: 3m 33s
2:	learn: 0.6254702	test: 0.6254702	best: 0.6254702 (2)	total: 2.83s	remaining: 4m 35s
0:	learn: 1.3774839	test: 1.3776056	best: 1.3776056 (0)	total: 2.77s	remaining: 12m 15s
0:	learn: 1.2656913	test: 1.2675127	best: 1.2675127 (0)	total: 2.66s	remaining: 12m 10s
3:	learn: 0.5993766	test: 0.5993766	best: 0.5993766 (3)	total: 3.22s	remaining: 3m 28s
0:	learn: 1.2164942	test: 1.2190159	best: 1.2190159 (0)	total: 3.22s	remaining: 13m 59s
3:	learn: 0.6119716	test: 0.6119716	best: 0.6119716 (3)	total: 3.62s	remaining: 4m 23s
0:	learn: 1.3140604	test: 1.3153717	best

[I 2024-05-02 11:06:40,959] Trial 7 finished with value: 0.5972232162026766 and parameters: {'iterations': 276, 'learning_rate': 0.34435411294384416, 'depth': 4, 'l2_leaf_reg': 5.660899573590611, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.2932951386061047}. Best is trial 7 with value: 0.5972232162026766.


256:	learn: 1.0122864	test: 1.0202017	best: 1.0202017 (256)	total: 12m 1s	remaining: 28.1s
144:	learn: 0.8878628	test: 0.9597436	best: 0.9594893 (138)	total: 12m 2s	remaining: 9m 42s
127:	learn: 0.8519132	test: 0.9669512	best: 0.9632117 (75)	total: 12m 2s	remaining: 11m 51s
180:	learn: 0.4079775	test: 0.4170635	best: 0.4170635 (180)	total: 12m 3s	remaining: 7m 35s
218:	learn: 0.4043156	test: 0.4171560	best: 0.4171560 (218)	total: 12m 3s	remaining: 2m 25s
154:	learn: 0.8951255	test: 0.9717891	best: 0.9711653 (143)	total: 12m 4s	remaining: 9m 48s
174:	learn: 0.9208912	test: 0.9609260	best: 0.9609260 (174)	total: 12m 4s	remaining: 8m 16s
257:	learn: 1.0120960	test: 1.0200177	best: 1.0200177 (257)	total: 12m 3s	remaining: 25.2s
219:	learn: 0.4042471	test: 0.4171712	best: 0.4171560 (218)	total: 12m 6s	remaining: 2m 21s
181:	learn: 0.4079115	test: 0.4170475	best: 0.4170475 (181)	total: 12m 6s	remaining: 7m 31s
145:	learn: 0.8875211	test: 0.9597277	best: 0.9594893 (138)	total: 12m 6s	remainin

[I 2024-05-02 11:07:07,893] Trial 6 finished with value: 0.5714115317636393 and parameters: {'iterations': 267, 'learning_rate': 0.018556195830088773, 'depth': 4, 'l2_leaf_reg': 5.673924558187408, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.7001296868231299}. Best is trial 7 with value: 0.5972232162026766.


227:	learn: 0.4036936	test: 0.4170420	best: 0.4170420 (227)	total: 12m 28s	remaining: 1m 54s
133:	learn: 0.8483527	test: 0.9672064	best: 0.9632117 (75)	total: 12m 30s	remaining: 11m 11s
161:	learn: 0.8924846	test: 0.9715627	best: 0.9711373 (158)	total: 12m 30s	remaining: 9m 11s
182:	learn: 0.9192598	test: 0.9604933	best: 0.9604933 (182)	total: 12m 30s	remaining: 7m 39s
151:	learn: 0.8854133	test: 0.9598954	best: 0.9594893 (138)	total: 12m 31s	remaining: 9m 3s
228:	learn: 0.4035148	test: 0.4169766	best: 0.4169766 (228)	total: 12m 31s	remaining: 1m 51s
188:	learn: 0.4073743	test: 0.4167787	best: 0.4167775 (187)	total: 12m 32s	remaining: 7m 2s
162:	learn: 0.8919609	test: 0.9714640	best: 0.9711373 (158)	total: 12m 33s	remaining: 9m 5s
183:	learn: 0.9189805	test: 0.9603287	best: 0.9603287 (183)	total: 12m 33s	remaining: 7m 34s
134:	learn: 0.8476552	test: 0.9673280	best: 0.9632117 (75)	total: 12m 34s	remaining: 11m 4s
229:	learn: 0.4034714	test: 0.4169718	best: 0.4169718 (229)	total: 12m 34s

[I 2024-05-02 11:08:28,152] Trial 0 finished with value: 0.5926429542689473 and parameters: {'iterations': 263, 'learning_rate': 0.15505141490022714, 'depth': 8, 'l2_leaf_reg': 7.942720069527143, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClassOneVsAll', 'colsample_bylevel': 0.3073941483875938}. Best is trial 7 with value: 0.5972232162026766.
[I 2024-05-02 11:11:46,272] Trial 4 finished with value: 0.60044371287483 and parameters: {'iterations': 295, 'learning_rate': 0.1719515289931053, 'depth': 6, 'l2_leaf_reg': 8.044000766847503, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.5122977034609364}. Best is trial 4 with value: 0.60044371287483.
[I 2024-05-02 11:12:24,411] Trial 5 finished with value: 0.5949330852358119 and parameters: {'iterations': 295, 'learning_rate': 0.12098514519748414, 'depth': 5, 'l2_leaf_reg': 6.124846031969793, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClassOneVsAll', 'colsample_bylevel': 0.6267425925964215}. Be

In [26]:
def objective(trial):
    params = {
        'iterations' : trial.suggest_int('iterations', 250, 400),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 10),
        'grow_policy' : trial.suggest_categorical('grow_policy', ["SymmetricTree", "Lossguide"]),
        'loss_function' :  trial.suggest_categorical('loss_function', ["MultiClass", "MultiClassOneVsAll"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0)
    }

    model = CatBoostClassifier(**params,  random_seed=42)
    model.fit(train_dataset, eval_set=eval_dataset)
    preds = model.predict(eval_dataset)
    y_pred = [int(val[0]) for val in preds]
    cat_f1 = f1_score(eval_dataset.get_label(), y_pred, average = 'micro')
    return cat_f1

sampler = TPESampler(seed=123)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=20, n_jobs=-1)

[I 2024-05-02 11:18:13,037] A new study created in memory with name: no-name-698c36e7-bc0a-4649-9089-90f5a0323d1f


0:	learn: 0.6182924	test: 0.6182924	best: 0.6182924 (0)	total: 742ms	remaining: 4m 40s
0:	learn: 0.6109282	test: 0.6109282	best: 0.6109282 (0)	total: 839ms	remaining: 4m 39s
0:	learn: 0.6526327	test: 0.6526327	best: 0.6526327 (0)	total: 875ms	remaining: 5m 7s
1:	learn: 0.5867136	test: 0.5867136	best: 0.5867136 (1)	total: 1.55s	remaining: 4m 52s
1:	learn: 0.5807766	test: 0.5807766	best: 0.5807766 (1)	total: 1.75s	remaining: 4m 50s
1:	learn: 0.6249888	test: 0.6249888	best: 0.6249888 (1)	total: 1.66s	remaining: 4m 51s
0:	learn: 1.2506938	test: 1.2524499	best: 1.2524499 (0)	total: 2.02s	remaining: 8m 36s
2:	learn: 0.5728623	test: 0.5728623	best: 0.5728623 (2)	total: 2.25s	remaining: 4m 41s
2:	learn: 0.5692178	test: 0.5692178	best: 0.5692178 (2)	total: 2.74s	remaining: 5m 1s
0:	learn: 0.6325891	test: 0.6329455	best: 0.6329455 (0)	total: 2.66s	remaining: 12m 23s
2:	learn: 0.6058838	test: 0.6058838	best: 0.6058838 (2)	total: 2.66s	remaining: 5m 10s
3:	learn: 0.5668105	test: 0.5668105	best: 0.

[I 2024-05-02 11:28:56,143] Trial 4 finished with value: 0.5993225029223025 and parameters: {'iterations': 257, 'learning_rate': 0.355297975517466, 'depth': 5, 'l2_leaf_reg': 9.511353586777945, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.16009056305271757}. Best is trial 4 with value: 0.5993225029223025.


183:	learn: 0.8961698	test: 0.9692299	best: 0.9689777 (155)	total: 10m 41s	remaining: 11m 44s
209:	learn: 0.3866637	test: 0.4123359	best: 0.4123338 (208)	total: 10m 40s	remaining: 3m 36s
115:	learn: 0.4123247	test: 0.4204915	best: 0.4204915 (115)	total: 10m 42s	remaining: 21m 51s
158:	learn: 0.4016482	test: 0.4190842	best: 0.4190608 (157)	total: 10m 42s	remaining: 14m 49s
108:	learn: 0.3996975	test: 0.4191095	best: 0.4191095 (108)	total: 10m 44s	remaining: 22m 9s
184:	learn: 0.8959452	test: 0.9692990	best: 0.9689777 (155)	total: 10m 44s	remaining: 11m 39s
210:	learn: 0.3864677	test: 0.4122578	best: 0.4122578 (210)	total: 10m 43s	remaining: 3m 33s
77:	learn: 0.4320948	test: 0.4353037	best: 0.4353037 (77)	total: 10m 45s	remaining: 35m 58s
116:	learn: 0.4122267	test: 0.4204810	best: 0.4204810 (116)	total: 10m 46s	remaining: 21m 43s
65:	learn: 0.9319964	test: 0.9672243	best: 0.9672243 (65)	total: 10m 47s	remaining: 45m 55s
211:	learn: 0.3864324	test: 0.4122604	best: 0.4122578 (210)	total: 

[I 2024-05-02 11:31:56,044] Trial 0 finished with value: 0.6036045707197214 and parameters: {'iterations': 281, 'learning_rate': 0.22218601272943603, 'depth': 7, 'l2_leaf_reg': 7.004874846301154, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClassOneVsAll', 'colsample_bylevel': 0.11413666751704385}. Best is trial 0 with value: 0.6036045707197214.


...

[I 2024-05-02 12:20:36,951] Trial 13 finished with value: 0.5969608053627233 and parameters: {'iterations': 376, 'learning_rate': 0.44902184158736846, 'depth': 6, 'l2_leaf_reg': 8.532906812014362, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.33043038577261485}. Best is trial 2 with value: 0.6037119206087932.


167:	learn: 0.3787105	test: 0.4132501	best: 0.4130397 (160)	total: 24m 52s	remaining: 14m 39s
151:	learn: 0.3528658	test: 0.4146708	best: 0.4136459 (92)	total: 24m 2s	remaining: 27m 50s
168:	learn: 0.3785210	test: 0.4132617	best: 0.4130397 (160)	total: 24m 59s	remaining: 14m 29s
39:	learn: 0.8584932	test: 0.9734422	best: 0.9715742 (35)	total: 16m 56s	remaining: 1h 52m 37s
34:	learn: 0.4072461	test: 0.4202807	best: 0.4202807 (34)	total: 13m 47s	remaining: 1h 44m
169:	learn: 0.3783510	test: 0.4132437	best: 0.4130397 (160)	total: 25m 6s	remaining: 14m 19s
152:	learn: 0.3524973	test: 0.4147882	best: 0.4136459 (92)	total: 24m 11s	remaining: 27m 40s
28:	learn: 0.4107258	test: 0.4216206	best: 0.4216206 (28)	total: 12m 47s	remaining: 2h 1m 21s
170:	learn: 0.3782414	test: 0.4132778	best: 0.4130397 (160)	total: 25m 11s	remaining: 14m 8s
153:	learn: 0.3522509	test: 0.4148212	best: 0.4136459 (92)	total: 24m 19s	remaining: 27m 28s
171:	learn: 0.3780833	test: 0.4132718	best: 0.4130397 (160)	total: 2

[I 2024-05-02 12:31:08,280] Trial 14 finished with value: 0.6047854194995109 and parameters: {'iterations': 267, 'learning_rate': 0.32892619389993455, 'depth': 7, 'l2_leaf_reg': 5.708497226938452, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClassOneVsAll', 'colsample_bylevel': 0.5496879953736925}. Best is trial 14 with value: 0.6047854194995109.


73:	learn: 0.3914115	test: 0.4147701	best: 0.4147701 (73)	total: 23m 59s	remaining: 1h 12m 56s
233:	learn: 0.3285522	test: 0.4181662	best: 0.4136459 (92)	total: 34m 23s	remaining: 13m 49s
63:	learn: 0.3943892	test: 0.4157110	best: 0.4157110 (63)	total: 23m 6s	remaining: 1h 26m 38s
234:	learn: 0.3283562	test: 0.4181970	best: 0.4136459 (92)	total: 34m 31s	remaining: 13m 39s
79:	learn: 0.7854772	test: 0.9853931	best: 0.9715742 (35)	total: 27m 19s	remaining: 1h 17m 11s
74:	learn: 0.3911777	test: 0.4147650	best: 0.4147650 (74)	total: 24m 11s	remaining: 1h 12m 14s
235:	learn: 0.3280556	test: 0.4181913	best: 0.4136459 (92)	total: 34m 37s	remaining: 13m 29s
236:	learn: 0.3275847	test: 0.4182243	best: 0.4136459 (92)	total: 34m 43s	remaining: 13m 20s
64:	learn: 0.3938792	test: 0.4155595	best: 0.4155595 (64)	total: 23m 19s	remaining: 1h 25m 47s
80:	learn: 0.7843596	test: 0.9856632	best: 0.9715742 (35)	total: 27m 33s	remaining: 1h 16m 33s
75:	learn: 0.3909214	test: 0.4146287	best: 0.4146287 (75)	t

[I 2024-05-02 12:40:36,363] Trial 16 finished with value: 0.6018154059018583 and parameters: {'iterations': 328, 'learning_rate': 0.29177899060018586, 'depth': 9, 'l2_leaf_reg': 5.4982862214588675, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClassOneVsAll', 'colsample_bylevel': 0.24647035641509324}. Best is trial 14 with value: 0.6047854194995109.


In [16]:
def objective(trial):
    params = {
        'iterations' : trial.suggest_int('iterations', 250, 400),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 10),
        'grow_policy' : trial.suggest_categorical('grow_policy', ["SymmetricTree", "Lossguide"]),
        'loss_function' :  trial.suggest_categorical('loss_function', ["MultiClass", "MultiClassOneVsAll"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0)
    }

    model = CatBoostClassifier(**params,  random_seed=42)
    model.fit(train_dataset, eval_set=eval_dataset)
    preds = model.predict(eval_dataset)
    y_pred = [int(val[0]) for val in preds]
    cat_f1 = f1_score(eval_dataset.get_label(), y_pred, average = 'micro')
    return cat_f1

sampler = TPESampler(seed=123, multivariate=True, group=True)
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=20, n_jobs=-1)

c:\Users\dcons\anaconda3\Lib\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
c:\Users\dcons\anaconda3\Lib\site-packages\optuna\samplers\_tpe\sampler.py:330: ExperimentalWarning: ``group`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-05-02 12:48:45,268] A new study created in memory with name: no-name-55151dae-924c-4f59-872a-a3b0d99e6139


0:	learn: 0.6024739	test: 0.6024739	best: 0.6024739 (0)	total: 419ms	remaining: 2m 5s
0:	learn: 0.6219450	test: 0.6220012	best: 0.6220012 (0)	total: 748ms	remaining: 4m 5s
0:	learn: 0.6046059	test: 0.6046059	best: 0.6046059 (0)	total: 543ms	remaining: 3m 27s
1:	learn: 0.5749497	test: 0.5749497	best: 0.5749497 (1)	total: 1.05s	remaining: 2m 35s
0:	learn: 1.2946279	test: 1.2964350	best: 1.2964350 (0)	total: 1.34s	remaining: 7m 46s
1:	learn: 0.5763190	test: 0.5763190	best: 0.5763190 (1)	total: 1.44s	remaining: 4m 34s
2:	learn: 0.5661788	test: 0.5661788	best: 0.5661788 (2)	total: 1.87s	remaining: 3m 5s
2:	learn: 0.5668394	test: 0.5668394	best: 0.5668394 (2)	total: 1.88s	remaining: 3m 57s
0:	learn: 1.3144700	test: 1.3161414	best: 1.3161414 (0)	total: 1.98s	remaining: 12m 2s
3:	learn: 0.5634670	test: 0.5634670	best: 0.5634670 (3)	total: 2.41s	remaining: 2m 58s
3:	learn: 0.5637417	test: 0.5637417	best: 0.5637417 (3)	total: 2.58s	remaining: 4m 4s
1:	learn: 0.5765428	test: 0.5771772	best: 0.577

[I 2024-05-02 12:57:18,874] Trial 3 finished with value: 0.5968892387700089 and parameters: {'iterations': 366, 'learning_rate': 0.46430195609631586, 'depth': 8, 'l2_leaf_reg': 7.7258748660826875, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.07568796078038316}. Best is trial 3 with value: 0.5968892387700089.


280:	learn: 0.3922090	test: 0.4149216	best: 0.4149216 (280)	total: 8m 31s	remaining: 1m 27s
224:	learn: 0.3885030	test: 0.4193759	best: 0.4193759 (224)	total: 8m 31s	remaining: 5m 58s
76:	learn: 0.3882027	test: 0.4157084	best: 0.4155827 (67)	total: 8m 32s	remaining: 20m 30s
338:	learn: 0.9105704	test: 0.9645596	best: 0.9645596 (338)	total: 8m 31s	remaining: 15.1s
277:	learn: 0.3888118	test: 0.4193615	best: 0.4192083 (259)	total: 8m 32s	remaining: 40.5s
281:	learn: 0.3921479	test: 0.4148947	best: 0.4148947 (281)	total: 8m 33s	remaining: 1m 25s
339:	learn: 0.9105281	test: 0.9645708	best: 0.9645596 (338)	total: 8m 32s	remaining: 13.6s
225:	learn: 0.3884162	test: 0.4193924	best: 0.4193759 (224)	total: 8m 33s	remaining: 5m 56s
278:	learn: 0.3888118	test: 0.4193623	best: 0.4192083 (259)	total: 8m 33s	remaining: 38.6s
340:	learn: 0.9102871	test: 0.9645862	best: 0.9645596 (338)	total: 8m 34s	remaining: 12.1s
60:	learn: 0.8733108	test: 0.9651361	best: 0.9651361 (60)	total: 8m 35s	remaining: 28m

[I 2024-05-02 12:57:36,637] Trial 7 finished with value: 0.5957441732865765 and parameters: {'iterations': 349, 'learning_rate': 0.3911101059392092, 'depth': 4, 'l2_leaf_reg': 7.753916498472062, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.1153240090328003}. Best is trial 3 with value: 0.5968892387700089.


...

[I 2024-05-02 13:36:08,295] Trial 14 finished with value: 0.5997638302440421 and parameters: {'iterations': 260, 'learning_rate': 0.32213821853252483, 'depth': 6, 'l2_leaf_reg': 7.610651246905016, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.4847154353298069}. Best is trial 9 with value: 0.6025787828908132.


364:	learn: 0.9034733	test: 0.9571112	best: 0.9567466 (349)	total: 38m 38s	remaining: 2m 32s
63:	learn: 1.0056790	test: 1.0152402	best: 1.0152402 (63)	total: 4m 16s	remaining: 13m 33s
50:	learn: 0.9781140	test: 0.9891912	best: 0.9891912 (50)	total: 4m 3s	remaining: 19m 9s
64:	learn: 1.0048701	test: 1.0146201	best: 1.0146201 (64)	total: 4m 20s	remaining: 13m 30s
51:	learn: 0.9769671	test: 0.9882522	best: 0.9882522 (51)	total: 4m 7s	remaining: 19m
133:	learn: 0.8785160	test: 0.9541810	best: 0.9537218 (116)	total: 27m 34s	remaining: 41m 46s
365:	learn: 0.9033219	test: 0.9572429	best: 0.9567466 (349)	total: 38m 45s	remaining: 2m 26s
177:	learn: 0.9359511	test: 0.9645922	best: 0.9645922 (177)	total: 32m 20s	remaining: 32m 20s
258:	learn: 0.8357800	test: 0.9491201	best: 0.9491108 (257)	total: 47m 21s	remaining: 24m 18s
44:	learn: 0.9003504	test: 0.9623166	best: 0.9623166 (44)	total: 12m 51s	remaining: 1h 10m 19s
65:	learn: 1.0041813	test: 1.0139894	best: 1.0139894 (65)	total: 4m 24s	remainin

[I 2024-05-02 13:38:47,035] Trial 8 finished with value: 0.5998115413058518 and parameters: {'iterations': 389, 'learning_rate': 0.411948478944992, 'depth': 4, 'l2_leaf_reg': 9.311074290201326, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.8452465581720933}. Best is trial 9 with value: 0.6025787828908132.


146:	learn: 0.8728124	test: 0.9534320	best: 0.9534320 (146)	total: 30m 1s	remaining: 38m 48s
14:	learn: 0.4345243	test: 0.4394352	best: 0.4394352 (14)	total: 2m 31s	remaining: 43m 41s
191:	learn: 0.9326302	test: 0.9629117	best: 0.9629117 (191)	total: 34m 45s	remaining: 29m 41s
271:	learn: 0.8310877	test: 0.9492560	best: 0.9491108 (257)	total: 49m 49s	remaining: 21m 58s
55:	learn: 0.8848529	test: 0.9608989	best: 0.9608877 (52)	total: 15m 20s	remaining: 1h 4m 21s
102:	learn: 0.9834743	test: 0.9955687	best: 0.9955687 (102)	total: 6m 51s	remaining: 10m 55s
90:	learn: 0.9559516	test: 0.9743691	best: 0.9743691 (90)	total: 6m 37s	remaining: 14m 37s
103:	learn: 0.9833031	test: 0.9953821	best: 0.9953821 (103)	total: 6m 55s	remaining: 10m 51s
91:	learn: 0.9553473	test: 0.9739427	best: 0.9739427 (91)	total: 6m 41s	remaining: 14m 32s
15:	learn: 0.4321157	test: 0.4372724	best: 0.4372724 (15)	total: 2m 38s	remaining: 42m 50s
92:	learn: 0.9549840	test: 0.9737515	best: 0.9737515 (92)	total: 6m 45s	rem

[I 2024-05-02 13:50:27,826] Trial 16 finished with value: 0.5870369045063097 and parameters: {'iterations': 267, 'learning_rate': 0.08076433960691354, 'depth': 4, 'l2_leaf_reg': 7.984374102721938, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.6794361832966413}. Best is trial 9 with value: 0.6025787828908132.
[I 2024-05-02 13:52:12,004] Trial 17 finished with value: 0.5967341778191274 and parameters: {'iterations': 292, 'learning_rate': 0.2013674812207718, 'depth': 4, 'l2_leaf_reg': 8.807928902032051, 'grow_policy': 'Lossguide', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.6523990395506246}. Best is trial 9 with value: 0.6025787828908132.
[I 2024-05-02 13:59:04,012] Trial 4 finished with value: 0.6092464037787161 and parameters: {'iterations': 392, 'learning_rate': 0.1636393093068129, 'depth': 8, 'l2_leaf_reg': 8.382816537101032, 'grow_policy': 'SymmetricTree', 'loss_function': 'MultiClass', 'colsample_bylevel': 0.43254443155292754}. Best is 

In [17]:
optuna.visualization.plot_parallel_coordinate(study)